In [43]:
import numpy as np
from keras.layers import Input, Dense, BatchNormalization, Activation, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.models import Model

In [15]:
def load_dataset(train_path, test_path):
    train_dataset = h5py.File(train_path, "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File(test_path, "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [23]:
import h5py
train_path = 'data/train_happy.h5'
test_path = 'data/test_happy.h5'
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset(train_path, test_path)

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


In [162]:
def HappyModel(input_shape):
    X_input= Input(input_shape)

    X = Conv2D(32, (3, 3), strides=(1, 1), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2))(X)    

    X = Flatten()(X)
    X = Dense(1, activation='sigmoid')(X)

    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    return model

In [164]:
happyModel = HappyModel(X_train.shape[1:])
happyModel.summary()

Model: "HappyModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)          │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_20               │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_26 (Activation)           │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_26 (MaxPooling2D)      │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_17 (Flatten)                 │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │          32,769 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,793 (132.00 KB)

 Trainable params: 33,729 (131.75 KB)

 Non-trainable params: 64 (256.00 B)

In [166]:
happyModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
happyModel.fit(x=X_train, y=Y_train, epochs=20, batch_size=50)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 165ms/step - accuracy: 0.5643 - loss: 2.3304
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.7918 - loss: 0.5813
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - accuracy: 0.8995 - loss: 0.2130
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.9481 - loss: 0.1437
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - accuracy: 0.9586 - loss: 0.0979
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 162ms/step - accuracy: 0.9839 - loss: 0.0654
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - accuracy: 0.9813 - loss: 0.0843
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.9868 - loss: 0.0623
Epoch 9/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 181ms/step - accuracy: 0.9873 - loss: 0.0500
Epoch 10/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - accuracy: 0.9786 - loss: 0.0647
Epoch 11/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - accuracy: 0.9863 - loss: 0.0538
Epoch 12/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step

In [167]:
preds = happyModel.evaluate(x=X_test, y=Y_test)

print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9502 - loss: 0.2747 
Loss = 0.2752242386341095
Test Accuracy = 0.95333331823349
